# Task Completion Analysis

Compute task completion metrics for Overcooked environments.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

In [ ]:
RESULTS_CSV = "/content/drive/MyDrive/results_combined_new.csv"
TASK_COMP_CSV = "/content/drive/MyDrive/task_completion.csv"

DISH_REWARD = 20
MAX_DISHES = 20

In [ ]:
df = pd.read_csv(RESULTS_CSV)
df_final = df[df["phase"] == "final"].copy()
df_final["env_clean"] = df_final["env"].str.strip()

In [ ]:
df_final["estimated_dishes"] = df_final["mean_return"] / DISH_REWARD
df_final["task_completion_rate"] = (df_final["estimated_dishes"] / MAX_DISHES).clip(0, 1)

In [ ]:
task_agg = df_final.groupby(["baseline", "env_clean"]).agg(
    tc_mean=("task_completion_rate", "mean"),
    tc_std=("task_completion_rate", "std"),
    dishes_mean=("estimated_dishes", "mean"),
    dishes_std=("estimated_dishes", "std")
).reset_index()

task_agg.to_csv(TASK_COMP_CSV, index=False)
print(task_agg.round(3))

In [ ]:
pivot = task_agg.pivot(index="baseline", columns="env_clean", values="tc_mean")

plt.figure(figsize=(10, 6))
pivot.plot(kind="bar", width=0.8, ax=plt.gca())
plt.ylabel("Task Completion Rate")
plt.xlabel("Baseline")
plt.ylim(0, 1)
plt.legend(title="Environment", bbox_to_anchor=(1.02, 1))
plt.tight_layout()
plt.show()

In [ ]:
heatmap_data = pivot[["No Noise", "Noise", "Delay", "Combo"]]

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="YlGnBu", vmin=0, vmax=1)
plt.title("Task Completion Rate by Baseline and Environment")
plt.tight_layout()
plt.show()

In [ ]:
overall = df_final.groupby("baseline").agg(
    overall_tc_mean=("task_completion_rate", "mean"),
    overall_tc_std=("task_completion_rate", "std")
).sort_values("overall_tc_mean", ascending=False)

print("Overall Task Completion Rankings:")
print(overall.round(3))